In [ ]:
 # Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Updated_Transformed_Student_Depression_Prediction.csv to Updated_Transformed_Student_Depression_Prediction.csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['Updated_Transformed_Student_Depression_Prediction.csv']))

In [ ]:
label_encoder = LabelEncoder()
df['DPN_State'] = label_encoder.fit_transform(df['DPN_State'])

In [ ]:
# Define input features and labels (targets)
X = df.drop(['Age_range', 'Score', 'DPN_State'], axis=1)  # Exclude these columns
y_score = df['Score']  # Predicting score
y_state = df['DPN_State']  # Predicting DPN_State

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_score_train, y_score_test, y_state_train, y_state_test = train_test_split(
    X, y_score, y_state, test_size=0.2, random_state=42
)

In [ ]:
# Normalize the features by dividing by the maximum value
X_train = X_train / X_train.max()
X_test = X_test / X_test.max()

In [ ]:
# Convert 'DPN_State' to categorical (for classification task)
y_state_train = to_categorical(y_state_train)
y_state_test = to_categorical(y_state_test)

In [ ]:
# Build the neural network model
# Define input layer
input_layer = Input(shape=(X_train.shape[1],))

In [ ]:
# Define shared layers (common layers for both tasks)
x = Dense(64, activation='relu')(input_layer)
x = Dense(32, activation='relu')(x)

In [ ]:
# Output 1: Predict 'Score' (Regression task)
score_output = Dense(1, name='score_output')(x)

In [ ]:
# Output 2: Predict 'DPN_State' (Classification task with softmax activation)
dpn_state_output = Dense(y_state_train.shape[1], activation='softmax', name='dpn_state_output')(x)


In [ ]:
# Create the model with two outputs
model = Model(inputs=input_layer, outputs=[score_output, dpn_state_output])

In [ ]:
# Compile the model with different losses for each output
model.compile(optimizer='adam',
              loss={'score_output': 'mse', 'dpn_state_output': 'categorical_crossentropy'},
              metrics={'score_output': 'mse', 'dpn_state_output': 'accuracy'})

In [ ]:
# Train the model
model.fit(X_train, {'score_output': y_score_train, 'dpn_state_output': y_state_train},
          epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - dpn_state_output_accuracy: 0.3780 - loss: 147.3040 - score_output_mse: 145.7902 - val_dpn_state_output_accuracy: 0.3162 - val_loss: 109.9348 - val_score_output_mse: 108.4463
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - dpn_state_output_accuracy: 0.3391 - loss: 131.9207 - score_output_mse: 130.4404 - val_dpn_state_output_accuracy: 0.3675 - val_loss: 86.4526 - val_score_output_mse: 85.0426
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - dpn_state_output_accuracy: 0.3605 - loss: 93.9252 - score_output_mse: 92.5207 - val_dpn_state_output_accuracy: 0.3675 - val_loss: 54.9456 - val_score_output_mse: 53.6299
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - dpn_state_output_accuracy: 0.3789 - loss: 62.0404 - score_output_mse: 60.6965 - val_dpn_state_output_accuracy: 0.4188 - val_loss: 23.4202 - val_score_output_mse: 22.1946
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - dpn_state_output_accuracy: 0.4832 - loss: 23.0804 - s

In [ ]:
# Save the trained model for future use
model.save('dpn_model_multioutput.keras')


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained model
model = load_model('dpn_model_multioutput.keras')

# Example input (ensure the input matches the structure of your features)
test_input = np.array([[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1]])  # Example input values

# Normalize the input
test_input = test_input / test_input.max()

# Make predictions
predictions = model.predict(test_input)

predicted_score = predictions[0]  # Score output
predicted_dpn_state = predictions[1]  # DPN_State output

# Convert predicted DPN_State back to the original labels (Normal, Mild, etc.)
dpn_state_label = label_encoder.inverse_transform([np.argmax(predicted_dpn_state)])

print(f'Predicted Score: {predicted_score[0]}')
print(f'Predicted DPN State: {dpn_state_label[0]}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Predicted Score: [2.0183353]
Predicted DPN State: Normal


In [ ]:
from google.colab import files

# Download the model to your local machine
files.download('dpn_model_multioutput.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>